In [81]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import json

import io
import re

In [82]:
url = 'https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A005930'

request = requests.get(url)
soup = BeautifulSoup(request.text, 'html.parser')

In [83]:
tables = soup.find_all('table')

market_conditions = tables[0]

In [87]:
df = pd.read_html(io.StringIO(str(market_conditions)))
original_dict = df[0].dropna().to_dict(orient='tight')
my_list = original_dict['data']
values_list = []
for text in my_list:
  first_values = text[1::2]
  final_values = []
  for value in first_values:
    if '/' in value:
      split_values = value.replace(' ', '').split('/')
      final_values.extend(split_values)
    else:
      final_values.append(value)
  values_list.append(final_values)
values = [item for sublist in values_list for item in sublist]
json_dict = {
    "종가(원)": values[0],
    "전일대비(원)": values[1],
    "수익률(%)": values[2],
    "거래량(주))": values[3],
    "최고가(52주)": values[4],
    "최저가(52주)": values[5],
    "거래대금(억원)": values[6],
    "수익률(1M)": values[7],
    "수익률(3M)": values[8],
    "수익률(6M)": values[9],
    "수익률(1Y)": values[10],
    "외국인지분율(%)": values[11],
    "시가총액(상장예정포함,억원)": values[12],
    "베타(1년)": values[13],
    "시가총액(보통주,억원)": values[14],
    "액면가(원)": values[15],
    "발행주식수(보통주)": values[16],
    "발행주식수(우선주)": values[17],
    "종가(NXT)": values[18],
    "유동주식수(주)": values[19],
    "유동비율(%)": values[20],
    "거래량(NXT, 주)": values[21],
    "거래대금(NXT,억원)": values[22],
}
market_status = json.dumps(json_dict, ensure_ascii=False, indent=4)

In [94]:
df = pd.read_html(io.StringIO(str(tables[1])))[0]
earning_issue = df.to_json(orient='records', force_ascii=False, indent=4)
print(earning_issue)

[
    {
        "잠정실적발표예정일":"미정",
        "예상실적(영업이익, 억원)":88501,
        "3개월전예상실적대비(%)":-5.84,
        "전년동기대비(%)":-3.63
    }
]


In [104]:
df = pd.read_html(io.StringIO(str(tables[2])))[0]
df.set_index('운용사명', inplace=True)
holdings_status = df.to_csv('005930_holdings_status.csv')

In [109]:
df = pd.read_html(io.StringIO(str(tables[3])))[0]
df.dropna(inplace=True)
df.set_index('항목', inplace=True)

                                        보통주    지분율       최종변동일
항목                                                            
삼성생명보험(외 15인)                   119256.6388  20.15  2025/09/01
국민연금공단                           45863.7667   7.75  2022/08/16
BlackRock Fund Advisors(외 15인)   30039.1061   5.07  2019/01/28
자사주                               8331.3157   1.41  2025/09/10 float64
